In [47]:
import pandas as pd
import numpy as np
import sys
import math
import matplotlib.pyplot as plt

In [48]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases
        
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)
        
class Activation_ReLu:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
        
        
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0
        
class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities
    
    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss
    
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)
        
        negative_log_likelihood = -np.log(correct_confidences)
        return negative_log_likelihood
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs/samples
        
class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()
        
    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs/ samples

In [49]:
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay 
        self.iterations = 0
        self.momentum = momentum
        
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = (self.learning_rate * 
                                           (1. / (1. + self.decay * self.iterations)))
        
    def update_params(self, layer):
        if self.momentum:
            
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            # Build weight updates with momentum
            weight_updates = (self.momentum * layer.weight_momentums - 
                            self.current_learning_rate * layer.dweights 
                            )
            layer.weight_momentums = weight_updates
            # Build bias updates      
            bias_updates = (self.momentum * layer.bias_momentums - 
                            self.current_learning_rate * layer.dbiases
                            )
            layer.bias_momentums = bias_updates
            
        else: 
            # Same as before adding momentum (check Spiral_Data.ipynb)
            weight_updates = -self.learning_rate * layer.dweights
            bias_updates += -self.learning_rate * layer.dbiases

        #Update the weights and biases
        layer.weights += weight_updates
        layer.biases += bias_updates
          
    def post_update_params(self):
        self.iterations += 1

In [50]:
def spiral_data(points, classes):
    X = np.zeros((points*classes, 2))
    y = np.zeros(points*classes, dtype='uint8')
    for class_number in range(classes):
        ix = range(points*class_number, points*(class_number+1))
        r = np.linspace(0.0, 1, points)  # radius
        t = np.linspace(class_number*4, (class_number+1)*4, points) + np.random.randn(points)*0.2
        X[ix] = np.c_[r*np.sin(t*2.5), r*np.cos(t*2.5)]
        y[ix] = class_number
    return X, y
X, y = spiral_data(100,3)


In [51]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLu()
# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
# Create optimizer
optimizer = Optimizer_SGD(decay=1e-3, momentum=0.5) # Train in loop
for epoch in range(10001):
  # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs

    dense2.forward(activation1.output)
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)
  # Calculate accuracy from output of activation2 and targets # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1) 
    accuracy = np.mean(predictions==y)

    if not epoch % 100: 
      print(f'epoch: {epoch}, ' +
            f'acc: {accuracy:.3f}, ' +
            f'loss: {loss:.3f}, ' +
            f'lr: {optimizer.current_learning_rate}')
  # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.360, loss: 1.099, lr: 1.0
epoch: 100, acc: 0.423, loss: 1.066, lr: 0.9099181073703367
epoch: 200, acc: 0.423, loss: 1.065, lr: 0.8340283569641367
epoch: 300, acc: 0.423, loss: 1.064, lr: 0.7698229407236336
epoch: 400, acc: 0.427, loss: 1.059, lr: 0.7147962830593281
epoch: 500, acc: 0.453, loss: 1.048, lr: 0.66711140760507
epoch: 600, acc: 0.450, loss: 1.031, lr: 0.6253908692933083
epoch: 700, acc: 0.437, loss: 1.009, lr: 0.5885815185403178
epoch: 800, acc: 0.460, loss: 0.987, lr: 0.5558643690939411
epoch: 900, acc: 0.443, loss: 0.991, lr: 0.526592943654555
epoch: 1000, acc: 0.440, loss: 0.985, lr: 0.5002501250625312
epoch: 1100, acc: 0.443, loss: 0.982, lr: 0.4764173415912339
epoch: 1200, acc: 0.463, loss: 0.974, lr: 0.45475216007276037
epoch: 1300, acc: 0.470, loss: 0.969, lr: 0.43497172683775553
epoch: 1400, acc: 0.483, loss: 0.961, lr: 0.4168403501458941
epoch: 1500, acc: 0.483, loss: 0.958, lr: 0.4001600640256102
epoch: 1600, acc: 0.510, loss: 0.951, lr: 0.38476337

In [52]:

class Optimizer_Adagrad:
    def __init__(self, learning_rate=1.0, decay=0.0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay 
        self.iterations = 0
        self.epsilon = epsilon
        
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = (self.learning_rate * 
                                           (1. / (1. + self.decay * self.iterations)))
        
    def update_params(self, layer):
        if not hasattr(layer, 'weight_momentums'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        
        # Update cache with squared current gradients
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2
        
        layer.weights += (-self.learning_rate * layer.dweights /
                          (np.sqrt(layer.weight_cache) + self.epsilon)
                        )
        layer.biases += (-self.learning_rate * layer.dbiases /
                          (np.sqrt(layer.bias_cache) + self.epsilon)
                        )
          
    def post_update_params(self):
        self.iterations += 1

In [57]:
class Optimizer_RMSprop:
    def __init__(self, 
                learning_rate=0.001, 
                decay=0.0, 
                epsilon=1e-7,
                rho=0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho
    
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = (self.learning_rate * 
                                          (1.0 /(1.0 + self.decay * self.iterations)))
    
    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        
        layer.weight_cache = (self.rho * layer.weight_cache +
                              (1.0 - self.rho) * layer.dweights**2)
        layer.bias_cache = (self.rho * layer.bias_cache +
                              (1.0 - self.rho) * layer.dbiases**2)
        
        layer.weights += (-self.current_learning_rate * 
                           layer.dweights / 
                           (np.sqrt(layer.weight_cache) + self.epsilon))
        
        layer.biases += (-self.current_learning_rate * 
                           layer.dbiases / 
                           (np.sqrt(layer.bias_cache) + self.epsilon))

    def post_update_params(self):
        self.iterations += 1
    

In [61]:
class Optimizer_Adam:
    def __init__(self,
                 learning_rate=0.001,
                 decay=0.0,
                 epsilon=1e-7,
                 beta_1=0.9,
                 beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        
    def pre_update_params(self): 
        if self.decay:
            self.current_learning_rate = (self.learning_rate * 
                                          (1.0 / (1.0 + self.decay * self.iterations)))
    
    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)
        # Update momentum  with current gradients
        layer.weight_momentums = (self.beta_1 * layer.weight_momentums +
                                  (1.0 - self.beta_1) * layer.dweights)
        layer.bias_momentums = (self.beta_1 * layer.bias_momentums +
                                  (1.0 - self.beta_1) * layer.dbiases)
        # Get corrected momentum
        weight_momentums_corrected = (layer.weight_momentums / 
                                      (1 - self.beta_1 ** (self.iterations + 1)))
        bias_momentums_corrected = (layer.bias_momentums / 
                                    (1 - self.beta_1 ** (self.iterations + 1)))
        
        # Update cache with squared current gradients
        layer.weight_cache = (self.beta_2 * layer.weight_cache + 
                              (1 - self.beta_2) * layer.dweights**2)
        layer.bias_cache = (self.beta_2 * layer.bias_cache + 
                              (1 - self.beta_2) * layer.dbiases**2)
        
        weight_cache_corrected = (layer.weight_cache / 
                                  (1-self.beta_2**(self.iterations +1)))
        bias_cache_corrected = (layer.bias_cache / 
                                  (1-self.beta_2**(self.iterations +1)))
        
        layer.weights += (-self.current_learning_rate * 
                           weight_momentums_corrected / 
                           (np.sqrt(weight_cache_corrected) +
                               self.epsilon))
        layer.biases += (-self.current_learning_rate * 
                           bias_momentums_corrected / 
                           (np.sqrt(bias_cache_corrected) +
                               self.epsilon))
        
    def post_update_params(self):
        self.iterations += 1
        


In [63]:
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLu()
# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
# Create optimizer
"""
  optimizer = Optimizer_Adagrad(decay=1e-4, ) 
  optimizer = Optimizer_RMSprop(learning_rate=0.02, decay=1e-5, rho=0.999)

"""
optimizer = Optimizer_Adam(learning_rate=0.05, decay=5e-7)
for epoch in range(10001):
  # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs

    dense2.forward(activation1.output)
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)
  # Calculate accuracy from output of activation2 and targets # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1) 
    accuracy = np.mean(predictions==y)

    if not epoch % 100: 
      print(f'epoch: {epoch}, ' +
            f'acc: {accuracy:.3f}, ' +
            f'loss: {loss:.3f}, ' +
            f'lr: {optimizer.current_learning_rate}')
  # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.317, loss: 1.099, lr: 0.05
epoch: 100, acc: 0.670, loss: 0.717, lr: 0.04999752512250644
epoch: 200, acc: 0.757, loss: 0.545, lr: 0.04999502549496326
epoch: 300, acc: 0.803, loss: 0.463, lr: 0.049992526117345455
epoch: 400, acc: 0.840, loss: 0.423, lr: 0.04999002698961558
epoch: 500, acc: 0.850, loss: 0.388, lr: 0.049987528111736124
epoch: 600, acc: 0.837, loss: 0.366, lr: 0.049985029483669646
epoch: 700, acc: 0.843, loss: 0.353, lr: 0.049982531105378675
epoch: 800, acc: 0.863, loss: 0.356, lr: 0.04998003297682575
epoch: 900, acc: 0.877, loss: 0.329, lr: 0.049977535097973466
epoch: 1000, acc: 0.890, loss: 0.321, lr: 0.049975037468784345
epoch: 1100, acc: 0.877, loss: 0.305, lr: 0.049972540089220974
epoch: 1200, acc: 0.883, loss: 0.297, lr: 0.04997004295924593
epoch: 1300, acc: 0.867, loss: 0.289, lr: 0.04996754607882181
epoch: 1400, acc: 0.863, loss: 0.288, lr: 0.049965049447911185
epoch: 1500, acc: 0.880, loss: 0.278, lr: 0.04996255306647668
epoch: 1600, acc: 0.883, lo